In [14]:
import torch
import torch.nn as nn
import numpy as np
import math

In [15]:
with open("../data/datasets/sonar.all-data") as all_data_file:
    lines = all_data_file.readlines()
    all_data = []
    labels = []
    for line in lines:
        line = line.strip().split(',')
        label = line.pop()
        #line = np.asarray(line, dtype=float)
        if label == "R":
            labels.append(0)
            all_data.append(line)
        elif label == "M":
            labels.append(1)
            all_data.append(line)
        else:
            pass
    all_data = np.asarray(all_data, dtype=float)
    labels = np.asarray(labels, dtype=float)


In [16]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        x[x<0] *= -1
        x = (x-np.min(x))/(np.max(x) - np.min(x))
        output_array.append(x)
    return np.asarray(output_array)
x_norm = normalise_2darray(all_data)

In [17]:
def train_test_split(input_data, input_labels):
    indices = np.random.permutation(input_data.shape[0])
    split_idx = math.floor(input_data.shape[0] * 0.9)
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    train_data, test_data = torch.from_numpy(input_data[train_idx,:]).type(torch.FloatTensor), torch.from_numpy(input_data[test_idx,:]).type(torch.FloatTensor)
    train_labels, test_labels = torch.from_numpy(input_labels[train_idx]).type(torch.FloatTensor), torch.from_numpy(input_labels[test_idx]).type(torch.FloatTensor)
    return train_data, test_data, train_labels, test_labels

In [18]:
train_data, test_data, train_labels, test_labels = train_test_split(x_norm, labels)

In [19]:
print(all_data)
print(labels)

[[0.02   0.0371 0.0428 ... 0.0084 0.009  0.0032]
 [0.0453 0.0523 0.0843 ... 0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 ... 0.0164 0.0095 0.0078]
 ...
 [0.0522 0.0437 0.018  ... 0.0138 0.0077 0.0031]
 [0.0303 0.0353 0.049  ... 0.0079 0.0036 0.0048]
 [0.026  0.0363 0.0136 ... 0.0036 0.0061 0.0115]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [20]:
class Network(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.network = self.__setup_network()

    def __setup_network(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def forward(self, input):
        return self.network(input)
    
    def fit(self, input, labels, n_epochs, loss_fn, optimizer, batch_size=10):
        for epoch in range(n_epochs):
            start = 0
            #for i in range(math.floor(input.shape[0]/batch_size)):
            #    X_batch = input[start:(start+batch_size)]
            #    Ybatch = labels[start:(start+batch_size)]
            #    start += batch_size
            #    pass
            prediction = self.forward(input)
            loss = loss_fn(prediction, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(f"Epoch {epoch}:\tloss:{loss}")
            
    


In [21]:
model = Network(input_size=60, output_size=1)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.9, betas=(0.8, 0.95))
model.train()
model.fit(input=train_data, labels=train_labels, n_epochs=100, loss_fn=loss_fn, optimizer=optimizer)

Epoch 0:	loss:0.6645808219909668
Epoch 1:	loss:0.6336681246757507
Epoch 2:	loss:0.604480504989624
Epoch 3:	loss:0.5780047178268433
Epoch 4:	loss:0.5535826086997986
Epoch 5:	loss:0.5440340638160706
Epoch 6:	loss:0.5436834692955017
Epoch 7:	loss:0.5439351797103882
Epoch 8:	loss:0.5444506406784058
Epoch 9:	loss:0.5450596213340759
Epoch 10:	loss:0.5476834774017334
Epoch 11:	loss:0.5481240749359131
Epoch 12:	loss:0.5454043745994568
Epoch 13:	loss:0.5453516840934753
Epoch 14:	loss:0.5454282760620117
Epoch 15:	loss:0.5466386079788208
Epoch 16:	loss:0.5453544855117798
Epoch 17:	loss:0.5452448129653931
Epoch 18:	loss:0.5451863408088684
Epoch 19:	loss:0.5451464653015137
Epoch 20:	loss:0.5451071262359619
Epoch 21:	loss:0.5450624227523804
Epoch 22:	loss:0.545005738735199
Epoch 23:	loss:0.54493248462677
Epoch 24:	loss:0.5448429584503174
Epoch 25:	loss:0.5447370409965515
Epoch 26:	loss:0.544613242149353
Epoch 27:	loss:0.5444712042808533
Epoch 28:	loss:0.5443148016929626
Epoch 29:	loss:0.544146478176

In [22]:
model.eval()
model.forward(test_data)

tensor([[0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015],
        [0.1015]], grad_fn=<AddmmBackward0>)